# Jupyter notebook for searching and downloading TSG files.

# Pip install librarys

In [ ]:
%pip install ipython
%pip install ipywidgets
%pip install ipyleaflet
%pip install -i https://test.pypi.org/simple/ auscopecat

In [ ]:
# Prevent exit crashes
class StopExecutionError(Exception):
    def _render_traceback_(self):
        return []

# User input for search

In [ ]:
#Get bbox for search
import re

import ipywidgets as widgets
from ipyleaflet import CircleMarker, GeomanDrawControl, Map, Rectangle, basemaps
from IPython.display import Markdown, display

from auscopecat.api import MAX_FEATURES
from auscopecat.nvcl import search_cql_tsg_df
from auscopecat.utils import download_url

AUSTRALIA_BBOX = {'north': -10.6681, 'east': 159, 'south': -43.6345, 'west': 110}
#Get BoreholeName for search
borehole_name_text = widgets.Text(value='05GJD001', placeholder='the Name to search.', description='Name:', layout=widgets.Layout(width='200px'))

display(Markdown('## Input a Name and BBox to search'))
provider_dd = widgets.Dropdown(options=['CSIRO', 'NSW','NT', 'QLD','SA','TAS','VIC','WA'], value='WA', description='Provider:', layout=widgets.Layout(width='200px'))
australia_btn = widgets.Button(description = 'Select Australia', button_style = '', tooltip = 'Reset bounds to Australia', icon = '')
search_btn = widgets.Button(description = 'Search', button_style = '', tooltip = 'Search Boreholes', icon = '')
download_btn = widgets.Button(description = 'Download', button_style = '', tooltip = 'Download TSG Files', icon = '')
north_field = widgets.BoundedFloatText(description = 'North', value = AUSTRALIA_BBOX['north'], min = -90.0, max = 90.0, style = {'font_size': '170%' }, layout=widgets.Layout(width='200px'))
west_field = widgets.BoundedFloatText(description = 'West', value = AUSTRALIA_BBOX['west'], min = -180.0, max = 180.0, style = {'font_size': '170%' }, layout=widgets.Layout(width='200px'))
south_field = widgets.BoundedFloatText(description = 'South', value = AUSTRALIA_BBOX['south'], min = -90.0, max = 90.0, style = {'font_size': '170%'}, layout=widgets.Layout(width='200px'))
east_field = widgets.BoundedFloatText(description = 'East', value = AUSTRALIA_BBOX['east'], min = -180.0, max = 180.0, style = {'font_size': '170%'}, layout=widgets.Layout(width='200px'))
input_box_layout = widgets.Layout(display = 'flex', flex_flow = 'row', align_items = 'stretch')
input_box = widgets.Box(children=[north_field, west_field, south_field, east_field, australia_btn], layout = input_box_layout)
input_search_layout = widgets.Layout(display = 'flex', flex_flow = 'row', align_items = 'stretch')
input_search = widgets.Box(children=[provider_dd, borehole_name_text, search_btn, download_btn], layout = input_search_layout)

display(Markdown('\n#### User Input Box'))
display(input_box)
display(input_search)

default_layout=widgets.Layout(width='1024px', height='768px')
m = Map(basemap=basemaps.Esri.WorldStreetMap, center = (-29.6, 133.0), zoom = 4,layout = default_layout)
rectangle = Rectangle(bounds=((AUSTRALIA_BBOX['south'],AUSTRALIA_BBOX['west']),(AUSTRALIA_BBOX['north'],AUSTRALIA_BBOX['east'])),opacity = 0.2)
# Add the rectangle to the map
m.add_layer(rectangle)

df = None
geometry_type = None
polygon = None
provider = 'WA'
selected_borehole_id =''

def on_provider_change(change):
    global provider
    if change['type'] == 'change' and change['name'] == 'value':
        provider =  change['new']
        display(Markdown(f'{provider}'))

provider_dd.observe(on_provider_change)

def reset_bounds(b):
    global geometry_type
    geometry_type = 'rectangle'
    north_field.value = AUSTRALIA_BBOX['north']
    south_field.value = AUSTRALIA_BBOX['south']
    west_field.value = AUSTRALIA_BBOX['west']
    east_field.value = AUSTRALIA_BBOX['east']

    rectangle = Rectangle(bounds=((AUSTRALIA_BBOX['south'],AUSTRALIA_BBOX['west']),(AUSTRALIA_BBOX['north'],AUSTRALIA_BBOX['east'])))
    # Add the rectangle to the map
    m.add(rectangle)

def download_tsg(b):
    global df
    try:
        for row in df.itertuples():
            url = row.DownloadLink
            if not url:
                continue
            fn = url.replace('/','-').replace(':','-')
            download_url(url,fn)
            display(Markdown(f'downloadTSG::downloaded: {fn}'))
        display(Markdown(f'Total TSG files downloaded:{df.shape[0]}'))
    except Exception as e:
        print(f'## Error download_tsg: {e}')
    return

def search_tsg(b):
    global df
    global polygon
    #Searching
    prov = provider
    bbox = f'{float(west_field.value):.2f}, {float(south_field.value):.2f}, {float(east_field.value):.2f}, {float(north_field.value):.2f}'
    name = f'{borehole_name_text.value}' #'Pilbara'
    kml_coords = polygon
    #clean all the markerLayers first
    for layer in m.layers:
        if isinstance(layer, CircleMarker):
            m.remove_layer(layer)

    try:
        df = None
        df = search_cql_tsg_df(prov, name, bbox, kml_coords, MAX_FEATURES)
    except Exception as e:
        print(f'Error querying data: {e}',500)
    if df is not None:
        display(Markdown(f'Total TSG files found :{df.shape[0]}'))
    else:
        raise StopExecutionError('Could not find any boreholes. Please change a name or bbox to search again!')

    #add circle-dot for nvcl boreholes:
    for index, row in enumerate(df.itertuples()):
        # if index > 10:
        #     break
        point = row[28]
        found = re.search(r'\((.+?)\)', point)
        if not found:
            continue
        cx, cy = found.group(1).split(' ')
        selected_borehole_id = f'{row[-1]}'#{row[1]}@{row[8]}@
        html = widgets.HTML()
        html.value = selected_borehole_id
        marker = CircleMarker(name = selected_borehole_id, location = (cy,cx), radius = 5, color = 'green', fillcolor='green')
        marker.on_click(create_popup_click(val1 = selected_borehole_id))
        m.add_layer(marker)
        marker.popup = html

def handle_draw(target, action, geo_json):
    global polygon
    global geometry_type
    if action == 'create' or action == 'drag':
        coords = geo_json[0]['geometry']['coordinates'][0]
        geometry_type =geo_json[0]['properties']['type']
        if geometry_type == 'rectangle':
            west_field.value = str(coords[0][0])
            south_field.value = str(coords[0][1])
            east_field.value = str(coords[2][0])
            north_field.value = str(coords[2][1])
            # TODO: Enable Rectangle control (possible?)
        elif geometry_type == 'polygon':
            polygonstr = ''
            for coord in coords:
                polygonstr += f'{coord[0]:.2f},{coord[1]:.2f} '
            polygon = polygonstr[0:-1]
            display(Markdown(f'{polygon}'))

    elif action == 'remove':
        # TODO: Disable Rectangle control (possible?)
        pass

def create_popup_click(val1):
    def popup_click(event, type, coordinates, val1 = val1):
        global selected_borehole_id
        selected_borehole_id = str(val1)
    return popup_click

australia_btn.on_click(reset_bounds)
search_btn.on_click(search_tsg)
download_btn.on_click(download_tsg)

draw_control = GeomanDrawControl()
draw_control.circlemarker = {}
draw_control.polyline = {}
draw_control.polygon = {"pathOptions": {"fillColor": "#6be5c3", "color": "#6be5c3", "fillOpacity": 0.2 }}
draw_control.rectangle = {'shapeOptions': {'fillColor': '#fca45d','color': '#fca45d','fillOpacity': 0.2}}
draw_control.on_draw(handle_draw)
m.add(draw_control)
m

In [ ]:
# show the search result in table df
import pandas as pd

pd.set_option('display.max_colwidth', None)

if df is not None:
    print(f'{df.shape=}')
else:
    raise StopExecutionError('Could not find any boreholes. Please change a name or bbox to search again!')
df['DownloadLink']